# Import Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import quantities as pq
# import elephant
import pandas as pd
# import statsmodels.api as sm
# from sklearn.decomposition import PCA
import neurotic
# from modules.utils import BehaviorsDataFrame

pq.markup.config.use_unicode = True  # allow symbols like mu for micro in output
pq.mN = pq.UnitQuantity('millinewton', pq.N/1e3, symbol = 'mN');  # define millinewton

# IPython Magics

In [ ]:
# make figures interactive and open in a separate window
# %matplotlib qt

# make figures interactive and inline
%matplotlib notebook

# make figures non-interactive and inline
# %matplotlib inline

# Data Parameters

In [ ]:
feeding_bouts = {
    'Branchy fresh food': ('IN VIVO / JG17 / 2019-08-20 / 004', [ 100, 300]), #[ 196,  296]),
    'Two-ply nori':       ('IN VIVO / JG17 / 2019-08-20 / 004', [2689, 2789]),
    'Tape nori':          ('IN VIVO / JG17 / 2019-08-17 / 001', [3532, 3705]), #[3532, 3632]),
}

# Import and Plot the Data

In [ ]:
sns.set(
#     context = 'poster',
    style = 'ticks',
    font_scale = 1,
    font = ['Palatino Linotype', 'serif'],
)

In [ ]:
# use Neo RawIO lazy loading to load much faster and using less memory
# - with lazy=True, filtering parameters specified in metadata are ignored
# - with lazy=True, loading via time_slice requires neo>=0.8.0
lazy = True

# load the metadata containing file paths
metadata = neurotic.MetadataSelector(file='../../data/metadata.yml')

last_data_set_name = None
for (food), (data_set_name, time_window) in feeding_bouts.items():

    # load the data
    metadata.select(data_set_name)
    if data_set_name is last_data_set_name:
        # skip reloading the data if it's already in memory
        pass
    else:
        blk = neurotic.load_dataset(metadata, lazy=lazy)
    last_data_set_name = data_set_name
    
    # grab the epochs
    peak_to_peak_force_epochs = next((ep for ep in blk.segments[0].epochs if ep.name == 'Peak-to-peak force period'), None)
    b10_to_b10_epochs = next((ep for ep in blk.segments[0].epochs if ep.name == 'B10-to-B10 motor pattern period'), None)
    assert peak_to_peak_force_epochs is not None
    assert b10_to_b10_epochs is not None
    
    # filter epochs by time window
    peak_to_peak_force_epochs = peak_to_peak_force_epochs.time_slice(time_window[0]*pq.s, time_window[1]*pq.s)
    b10_to_b10_epochs = b10_to_b10_epochs.time_slice(time_window[0]*pq.s, time_window[1]*pq.s)
    assert peak_to_peak_force_epochs.size == b10_to_b10_epochs.size
    
    # plot swallow durations
    plt.figure(figsize=(4,4))
    x = peak_to_peak_force_epochs.durations
    y = b10_to_b10_epochs.durations
    max_value = 15 #max(x.max().magnitude, y.max().magnitude)*1.1
    plt.scatter(x, y, facecolors='none', edgecolors='r')
    plt.plot([0, max_value], [0, max_value], ls=':', c='gray', zorder=0)
    plt.xlim([0, max_value])
    plt.ylim([0, max_value])
    plt.xlabel('Swallow duration (s)\nMeasured between force peaks')
    plt.ylabel('Swallow duration (s)\nMeasured between B10 burst starts')
    plt.title(food)
    plt.tight_layout()